## Generate Fingerprints for AF2 Structures & Cluster

In [26]:
import os
import sys
from ast import literal_eval
import json
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

#for pocket sequence alignment
from Bio.PDB.PDBParser import PDBParser
from Bio import pairwise2
from Bio.PDB import PDBIO
from Bio.PDB.PDBIO import Select
from Bio.Seq import Seq 
from Bio.pairwise2 import format_alignment 

#kissim
from kissim.encoding import Fingerprint

#for t_SNE
from sklearn.manifold import TSNE


/Users/carmenmasri/opt/anaconda3/envs/kissim/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


### Inputs/Outputs & Parameters

In [34]:
#metadata files
af_predictions = pd.read_csv('inputs/alphafold_predictions_sample.csv')[['pdb_id','Class']]
KLIFS_metadata = pd.read_csv('inputs/kinases_all.csv')[['group','family','pocket','uniprot','name','species']]

input_structs = 'inputs/AF_predictions' #input AF2 structures
output_pocket = 'outputs/AF_pockets' #output pdb files containing pocket residues
output_fingerprints = 'outputs/fingerprints_AF_all.csv' #output file containing kissim fingerprints

#t-SNE parameters
n_iters=[1000] #iterations
perplexities=[30] #perplexity

### Functions to extract pocket residues from structure file

In [8]:
#convert 3-letter amino acids to 1
def three_to_one(three_letter_code):
    '''converts any three letter code 
       to a one letter code'''
    mapping = {'Ala':'A',
               'Arg':'R',
               'Asn':'N',
               'Asp':'D',
               'Cys':'C',
               'Gln':'Q',
               'Glu':'E',
               'Gly':'G',
               'His':'H',
               'Ile':'I',
               'Leu':'L',
               'Lys':'K',
               'Met':'M',
               'Phe':'F',
               'Pro':'P',
               'Ser':'S',
               'Thr':'T',
               'Trp':'W',
               'Tyr':'Y',
               'Val':'V',}
    return mapping[three_letter_code[0].upper() 
                   + three_letter_code[1:].lower()]

In [9]:
#select pocket residues from structure
class Select_pocket(Select):
    def __init__(self, idx):
        self.idx = idx

    def accept_residue(self, residue):
            exist_count = idx.count(residue.get_id()[1])

            if exist_count != 0:
                return True
            else:
                return False

### Create DataFrame with all info on kinase + path to structure

In [13]:
af_predictions['Entry'] = af_predictions.apply(lambda x: x['pdb_id'].split('-')[1],axis=1)
af_predictions['pdb_path'] = af_predictions.apply(lambda x: f'{input_structs}/{x.pdb_id}.atp.pdb',axis=1)

#merge everything
af_data = pd.merge(af_predictions,KLIFS_metadata,left_on='Entry',right_on='uniprot',how='inner').set_index('uniprot')


### Extract pocket residues from AF2 structures

The following block reads the PDB file and:
* extracts contents and writes into string (to read as input in KiSSim)
* extracts sequence + resid (to align with uniprot pocket sequence)
* aligns the pdb residues to the uniprot pocket sequence
* extracts residues from PDB corresponding to pocket residues

In [24]:
complex_string = []
pocket_idx = []
all_seq = []

bad_paths=[] #store paths of structures that failed for debugging purposes
no_complex=[] #store structures that failed for debugging purposes

for idx,row in af_data.iterrows():
    pdb_path=row.pdb_path
    uniprot_pocket = row.pocket
    pdb_id = row.pdb_id
    
    try:
        #write complex file content into string
        f = open(pdb_path, "r")
        complex_string.append(f.read())
        f.close() 
    except:
        no_complex.append(pdb_path)
        continue
    
    #extract sequence + resid using biopython
    pdbparser = PDBParser()   #Parser for PDB file
    structure = pdbparser.get_structure('0', pdb_path) #Extract structure
    
    try:
        chain = next(structure.get_chains()) #select 1st chain (which contains protein)
        resid_pdb = []; res_pdb = []
        for residue in chain: #loop over residues in the chain
            if residue.id[0] == " ":
                resid_pdb.append(residue.id[1]) #select residues without the hetero flag (they belong to the protein)
                res_pdb.append(residue.resname)
                res1_pdb = [three_to_one(res) for res in res_pdb]

        #align to uniprot
        seq_PDB = Seq(''.join(res1_pdb))
        seq_uniprot = Seq(uniprot_pocket)

        alignments = pairwise2.align.globalms(seq_PDB, seq_uniprot,2, -2, -5, 0, one_alignment_only=True) 
                
        #find index of matched residue 
        alignment = alignments[0]
        matches = []
        j=0
        for i, (a, b) in enumerate(zip(alignment[0], alignment[1])):
            if a != '-' and b != '-': #includes mutations
                matches.append(j)
            else: 
                if a == '-':
                    j = j - 1
                    matches.append('-')
            j = j + 1
                    
        #resid of the pocket residues 
        idx = [match + resid_pdb[0] if match != '-' else '-' for match in matches]
        
        #if pocket residue missing, replace with negative index
        #(will be transformed to NaN when fingerprints are generated)
        nan_idx = [-1*i for i in range(1,86)] 
        pocket_idx = [nonzero_idx if nonzero_idx != '-' else nan for nonzero_idx,nan in zip(idx,nan_idx)]
        
        all_seq.append([uniprot_pocket[j]  if idx[j] != '-' else '-' for j in range(len(idx))])
        
        #extract these residues into pocket.pdb
        io = PDBIO()
        io.set_structure(structure)
        io.save(f'{output_pocket}/{pdb_id}_pocket.pdb', Select_pocket(pocket_idx))
        
        #write complex file content into string
        f = open(f'{output_pocket}/{pdb_id}_pocket_idx.txt', "w")
        f.write(' '.join(map(str,pocket_idx)))
        f.close()
    
    except StopIteration:
        bad_paths.append(path)
        


### Generate KiSSim Fingerprints

In [28]:
#get fingerprints
fingerprints_all = pd.DataFrame([])
fingerprints_all['uniprot'] = af_data['Entry']
fingerprints_all['pdb_id'] = af_data['pdb_id']

residue_ids = list(range(1,86))
bad_id=[]
for idx, row in af_data.iterrows():
        pdb_path = row.pdb_path
        uniprot = row.Entry
        pdb_id = row.pdb_id
        kinase_name = row.name
        
        try:
            #read complex as string
            f = open(pdb_path, "r")
            complex_string = f.read()
            f.close() 

            #read uniprot pocket sequence
            f = open(f'{output_pocket}/{pdb_id}_pocket_idx.txt', "r")
            uniprot_string = list(f.read())
            f.close()

            #read uniprot pocket sequence
            f = open(f'{output_pocket}/{pdb_id}_pocket_idx.txt', "r")
            pocket_idx = f.read().split(' ')
            f.close()

            nan_idx = [-1*i for i in range(1,86)] 
            pocket_idx = [int(nonzero_idx) if nonzero_idx != '-' else nan for nonzero_idx,nan in zip(pocket_idx,nan_idx)]
            #generate fingerprints
            fingerprint = Fingerprint.from_text(complex_string,'pdb', pocket_idx,residue_ids,pdb_id,kinase_name)

            #extract physicochemical features
            for key in fingerprint.physicochemical:
                feature_list = fingerprint.physicochemical[key].tolist()

                if key == 'sco': #replace values of G residues with 0 (no sidechain)
                    feature_list = [feature if res != 'G' else 0.0 for feature, res in zip(feature_list, uniprot_string)]


                fingerprints_all.loc[idx,key] = str(feature_list)

            #extract distance features
            for key in fingerprint.distances:
                feature_list = fingerprint.distances[key].tolist()
                fingerprints_all.loc[idx,key] = str(feature_list)
                
        except:
            print(pdb_path)
            continue
fingerprints_all.to_csv(output_fingerprints,index=False)


### Perform t-SNE on fingerprints

#### Re-arrange dataframe to use as input for t-SNE

In [33]:
#load dataset
fingerprints_all = pd.read_csv(output_fingerprints).drop_duplicates(subset=['uniprot'])

distance_keys = ['hinge_region','dfg_region','front_pocket','center']
dataset_all = pd.DataFrame([],index=fingerprints_all['uniprot'].values)
for key in distance_keys:    
    #replace nan in string with -999 for literal eval to work. nan will be dropped
    fingerprints_all = fingerprints_all[~fingerprints_all.isnull().any(1)]
    feature_list = fingerprints_all[key].str.replace('nan','-999').apply(literal_eval).tolist()
    data_df = pd.DataFrame(feature_list,index=fingerprints_all['uniprot'].values)
    
    #get rid of nan (set as -999 in the dataframe) by removing column 50 and then every row with nan 
    data_df = data_df.drop(49,axis=1)
    data_df = data_df[~data_df.isin([-999]).any(axis=1)]
    data_df.columns = [f'{key}_{i}' for i in range(len(data_df.columns)) ]
    
    dataset_all = pd.concat([dataset_all,data_df],axis=1)

dataset_all = dataset_all[~dataset_all.isnull().any(1)]

dataset_all

hinge_region_0  hinge_region_1  hinge_region_2  hinge_region_3  \
A0AUV4       12.957722       11.532850        9.199569       10.320819   
E9Q3S4       12.599704       11.122005        8.562771        9.666923   
O00141       12.859444       11.741398        9.466249        9.659949   
O00238       12.744567       11.632812        9.661626       10.017544   
O00444       13.321777       11.816965        9.452351       10.225027   
O00506       13.128567       11.634107        9.521685        9.740628   
O08605       12.456075       11.480284        8.626925        9.754073   
O08644       13.519521       12.140714        9.986884       10.417194   
O08648       13.080647       11.841934        9.832738       10.031174   
O08901       12.833611       11.331615        9.006399       10.028090   
O08911       12.647046       11.803310        9.508187       10.400660   
O09110       13.150115       11.937845        9.633609       11.061992   

        hinge_region_4  hinge_region_5  hinge_region_6  hinge_region_7  \
A0AUV4       12.915934       13.414791       16.851301       15.877371   
E9Q3S4       12.071548       13.286847       15.693749       13.050849   
O00141       11.933728       12.252559       15.561074       15.422397   
O00238       12.241053       12.436078       15.189171       14.017797   
O00444       12.421671       13.319070       16.644918       15.977077   
O00506       11.359278       10.428937       13.467441       13.751627   
O08605       12.423694       14.193515       17.359634       16.139620   
O08644       11.555800       10.266543       12.930622       12.950206   
O08648       12.420977       13.629883       15.768388       13.304367   
O08901       12.159077       13.459887       16.391365       14.948925   
O08911       12.050609       13.127879       15.757244       13.811452   
O09110       12.538393       12.400025       15.532213       14.491349   

        hinge_region_8  hinge_region_9  ...  center_74  center_75  center_76  \
A0AUV4       12.287630       11.048027  ...   8.632272   9.652345  13.227513   
E9Q3S4       12.636291       10.806272  ...   8.778232   9.041656  12.265953   
O00141       13.365890       11.000291  ...   8.608176   8.865643  12.008187   
O00238       12.767523       10.888959  ...   8.511335   8.917744  12.424570   
O00444       12.510734       11.031666  ...   8.058205   7.800839  11.154028   
O00506       12.406677       10.809791  ...   8.264639   7.971156  11.215286   
O08605       12.438636       10.738298  ...   8.674101   8.669357  12.052257   
O08644       12.385365       11.421936  ...   8.031147   7.663807  11.014686   
O08648       12.670934       11.122890  ...   8.428858   8.826139  12.221377   
O08901       11.526276       10.553385  ...   7.622121   7.824901  10.793247   
O08911       11.807184       10.516343  ...   7.990821   7.861681  11.522541   
O09110       12.453666       10.993541  ...   8.888996   9.000965  12.422392   

        center_77  center_78  center_79  center_80  center_81  center_82  \
A0AUV4   6.203915   2.763013   2.850827   5.273843   7.367781  10.155950   
E9Q3S4   6.370408   2.676410   2.958014   5.504191   7.935563   9.841313   
O00141   6.306172   2.611872   3.204777   5.635829   8.004719  10.532525   
O00238   6.702823   3.129531   3.531132   5.595687   7.854023  10.627388   
O00444   6.545267   2.952552   3.814662   6.542552   8.608204  11.057873   
O00506   6.447140   2.680721   3.217197   6.377535   9.316099  12.170013   
O08605   6.447325   2.784964   3.574779   6.728322   9.739150  11.317001   
O08644   6.941836   3.384619   4.436922   6.746827   9.815763   9.960793   
O08648   6.848302   3.028425   2.490175   4.937348   7.093876   9.628140   
O08901   5.216557   1.815317   4.049030   6.323332   9.062866  11.276818   
O08911   6.410044   2.821352   3.775828   6.681598   8.626925  11.224258   
O09110   6.999342   3.228290   3.318382   6.038461   8.462703   8.726471   

        center_83  
A0AUV4  10.244872  
E9Q3S4  10.3

#### Perform t-SNE

In [35]:
#perform t-SNE (3D)

for n_iter in n_iters:
    for perplexity in perplexities:
        n_components = 3
        tsne = TSNE(n_components,learning_rate='auto',n_iter=n_iter, perplexity=perplexity)
        tsne_result = tsne.fit_transform(dataset_all)
        tsne_result.shape
        #plot results
        tsne_result_df = pd.DataFrame({'tsne_1': tsne_result[:,0], 'tsne_2': tsne_result[:,1],'tsne_3': tsne_result[:,2], 'structure_ID':dataset_all.index.tolist()})
        tsne_result_df.set_index('structure_ID',inplace=True)
        tsne_result_df=pd.merge(af_data,tsne_result_df,left_index=True,right_index=True,how='right')
        tsne_result_df.reset_index(inplace=True)

        family_fig = px.scatter_3d(tsne_result_df, x='tsne_1', y='tsne_2', z='tsne_3',
                                   color="family", symbol="group",title=f't-SNE ({n_iter} iterations)',
                                   hover_data=['name','pdb_id','Class'])

        fig1 = px.scatter_3d(tsne_result_df, x='tsne_1', y='tsne_2', z='tsne_3',
                             color="family", symbol="group",title='t-SNE',
                            hover_data=['name','pdb_id','Class'])


        #figure labeled by kinase group
        group_fig = px.scatter_3d(tsne_result_df, x='tsne_1', y='tsne_2', z='tsne_3',
                                 color="group", title='distance to ' + key,
                                 hover_data=['name','pdb_id','Class'])
        for i in range(len(group_fig.data)):
            fig1.add_trace(group_fig.data[i])

        #figure labeled by kinase configuration
        conf_fig = px.scatter_3d(tsne_result_df, x='tsne_1', y='tsne_2', z='tsne_3',
                                 color="Class", title='distance to ' + key,
                                 hover_data=['name','pdb_id','Class'])
        for i in range(len(conf_fig.data)):
            fig1.add_trace(conf_fig.data[i])

        legends_total=len(family_fig.data)+len(group_fig.data)+len(conf_fig.data)

        updatemenus=[dict(type = "buttons", direction = "left",
                     buttons=list([
                     dict(args=[{'visible': [True if i < len(family_fig.data) else False for i in range(legends_total)]} ,],
                          label = "kinase family", method="update"),

                     dict(args=[{'visible': [True if i >= len(family_fig.data) and i < len(family_fig.data) + len(group_fig.data) else False for i in range(legends_total)]} ,],
                         label = "kinase group", method="update"),

                     dict(args=[{'visible': [True if i >= len(family_fig.data) + len(group_fig.data) and i < len(family_fig.data) + len(group_fig.data) + len(conf_fig.data) else False for i in range(legends_total)]} ,],
                          label = "kinase conformation", method="update"),])),]

        fig1.update_layout(updatemenus = updatemenus,
                          legend_title_text='')
        fig1.update_traces(marker_size=5)
        fig1.update_layout(legend=dict(title_font_family="Times New Roman",
                                  font=dict(size= 20)
        ))
        fig1.update_layout(legend= {'itemsizing': 'constant'})
    
        fig1.show()
        fig1.write_html(f'outputs/t-SNE-iter{n_iter}-perp{perplexity}.html')

/Users/carmenmasri/opt/anaconda3/envs/kissim/lib/python3.8/site-packages/sklearn/manifold/_t_sne.py:780: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  warnings.warn(
